# Imports

In [1]:
import pandas as pd
import polars as pl
import numpy as np
from utilities.schema import contracts_schema
from utilities.funcoes import prepro_obj

c:\Users\carol\Documents\bolsa\RISKGUARD\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Read Data

In [2]:
single_biding_indicator = pl.read_ipc("../../data/indicators/indicator_1016.arrow")

In [21]:
bench = pl.read_csv("../../data/benchmark.csv", schema_overrides={"N.º Procedimento (ID BASE)": pl.Utf8 
                                                                  , "N.º Contrato": pl.Utf8 })
pos_tagging = pl.read_csv("../../data/resultados_pos_tagging.csv", schema_overrides={"N.º Procedimento (ID BASE)": pl.Utf8 
                                                                  , "N.º Contrato": pl.Utf8 })

In [22]:
bench = bench.unique(["N.º Procedimento (ID BASE)",	"N.º Contrato"]).join(pos_tagging.select(["n_palavras_total", "NOUN", "ADP", "N.º Procedimento (ID BASE)", "N.º Contrato"]), on=["N.º Procedimento (ID BASE)", "N.º Contrato"], coalesce=True)

In [ ]:
# contratos_object = pl.read_parquet(
#     "../../data/contratos_cleaned_prepro.parquet",
#     columns=[
#         "N.º Procedimento (ID BASE)", "N.º Contrato", "Tipo(s) de contrato_LIMPO", "Objeto_LIMPO", "Objeto_LIMPO_2", "Objeto", "Tipo de procedimento"
#     ]
#     )

In [23]:
contratos_raw = pl.read_csv("../../data/impic_data/contratos.csv", separator=";", schema_overrides=contracts_schema(),
    columns=[
       "Objeto", "Tipo(s) de contrato", "Data da decisão adjudicação", "N.º Procedimento (ID BASE)", 
       "Data Decisão Contratar", "Data Celebração", "N.º Contrato", "Contratação Excluída", "Entidade(s) Adjudicante(s) - NIF",
       "Preço BASE (€)", "Preço Contratual (€)", "Local de execução das principais prestações objeto do contrato", "Tipo de procedimento"
    ], 
    null_values=["NULL"]) \
    .with_columns(
        pl.col("Data da decisão adjudicação").replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F"),
        pl.col("Data Decisão Contratar").replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F"),
        pl.col("Data Celebração").replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F")
    ) \
    .with_columns(pl.col("Data Celebração").dt.year().alias("ano_celebracao")) \
    .filter(pl.col("Contratação Excluída") == False).unique(subset=["N.º Procedimento (ID BASE)", "N.º Contrato"])

In [ ]:
final = bench.join(contratos_raw, on=["N.º Procedimento (ID BASE)", "N.º Contrato"], coalesce=True).to_pandas()
final["log_preco"] = np.log1p(final["Preço Contratual (€)"])
final = final.rename(columns={
    "Tipo(s) de contrato": "contract_type"
})

In [49]:
formula = """
num_bidders ~
    n_palavras_total +
    log_preco +
    C(contract_type) +
    C(ano_celebracao)
"""

In [50]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model_nb = smf.glm(
    formula=formula,
    data=final,
    family=sm.families.NegativeBinomial()
).fit(cov_type="HC3")

c:\Users\carol\Documents\bolsa\RISKGUARD\.venv\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [ ]:
print(model_nb.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:            num_bidders   No. Observations:                21106
Model:                            GLM   Df Residuals:                    21093
Model Family:        NegativeBinomial   Df Model:                           12
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -55859.
Date:                Wed, 04 Feb 2026   Deviance:                       9416.0
Time:                        17:47:02   Pearson chi2:                 1.28e+04
No. Iterations:                     6   Pseudo R-squ. (CS):            0.02371
Covariance Type:                  HC3                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [6]:
contratos_prepro = contratos_object.join(contratos_raw, how="left", on=["N.º Procedimento (ID BASE)", "N.º Contrato"], coalesce=True)

In [8]:
# resultados_pdf = pl.read_csv("../../data/results_reading_pdfs.csv",
#                              schema_overrides={
#                                  "identifier": pl.Utf8
#                              },
#                              columns=[
#                                  'identifier', 'caderno_pdf', 'caderno_pages', 'caderno_match_count', 'caderno_has_especificacoes_tecnicas'
#                              ]).rename({
#                                  "identifier": "N.º Procedimento (ID BASE)"
#                              })

In [8]:
objeto_indicator_a = pl.read_ipc("../../data/indicators/1011_a.arrow")
objeto_indicator_model = pl.read_ipc("../../data/indicators/1011_model.arrow")

In [9]:
objeto_indicator_model = objeto_indicator_model.select(['N.º Procedimento (ID BASE)',
 'N.º Contrato', 'model_confidence_logistic',
 'model_prediction_logistic',
 'flag_1011_model_logistic',
 'model_confidence_xgboost',
 'model_prediction_xgboost',
 'flag_1011_model_xgboost'])

In [15]:
# não esquecer que este indicador está apenas calculado para os contratos que fazem parte do dataset de teste

full_object_indicator = objeto_indicator_model.join(objeto_indicator_a, 
                                                on=['N.º Procedimento (ID BASE)', 'N.º Contrato'],
                                                coalesce=True)

In [16]:
objeto_indicator_model.shape

(187515, 8)

In [17]:
full_object_indicator.shape

(187515, 11)

## Número de palavras

In [12]:
# contratos_prepro = resultados_pdf.join(contratos_prepro, how="left", 
#                                                on="N.º Procedimento (ID BASE)", coalesce=True) \
#                                                .with_columns(
#                                                    pl.when(pl.col("caderno_pdf").is_not_null()).then(pl.lit(0)).otherwise(pl.lit(1)).alias("has_caderno")
#                                                )

In [18]:
contrats_analise = full_object_indicator.join(contratos_prepro, how="left", on=["N.º Procedimento (ID BASE)", "N.º Contrato"], coalesce=True) \
                                        .join(single_biding_indicator, how="left",
                                              on = ["N.º Procedimento (ID BASE)", "N.º Contrato"], coalesce=True) \
                                              .with_columns(
                                                    # data e mês de celebração
                                                    pl.col("Data Celebração").dt.year().alias("ano_celebracao"),
                                                    pl.col("Data Celebração").dt.month().alias("mes_celebracao"),
                                                    # corrigir o indicador sinigle bidding
                                                    pl.when(pl.col('indicator_1016')==0).then(pl.lit(1)).otherwise(pl.lit(0)).alias("indicator_1016_contrario")
                                                    # ver o número de páginas do caderno de encargos
                                                    #pl.when(pl.col("caderno_pages").is_null()).then(pl.lit(0)).otherwise(pl.col("caderno_pages")).alias("caderno_pages"),
                                              )


In [ ]:
contrats_analise.shape

In [ ]:
# contrats_analise = contratos_prepro.join(full_object_indicator, how="left", on=["N.º Procedimento (ID BASE)", "N.º Contrato"], coalesce=True)\
#                                                 .join(single_biding_indicator, how="left", 
#                                                  on = ["N.º Procedimento (ID BASE)", "N.º Contrato"], coalesce=True) \
#                                                  .with_columns(
#                                                      # data e mês de celebração
#                                                     pl.col("Data Celebração").dt.year().alias("ano_celebracao"),
#                                                     pl.col("Data Celebração").dt.month().alias("mes_celebracao"),
#                                                     # corrigir o indicador sinigle bidding
#                                                     pl.when(pl.col('indicator_1016')==0).then(pl.lit(1)).otherwise(pl.lit(0)).alias("indicator_1016_contrario")
#                                                     # ver o número de páginas do caderno de encargos
#                                                     #pl.when(pl.col("caderno_pages").is_null()).then(pl.lit(0)).otherwise(pl.col("caderno_pages")).alias("caderno_pages"),
#                                                     )

In [9]:
# contrats_analise = contrats_analise.filter(pl.col("flag_1011").is_not_null())

In [10]:
# contrats_analise = contrats_analise.with_columns((pl.col("Data Celebração").dt.year().alias("ano_celebracao")),
#                                                  (pl.col("Data Celebração").dt.month().alias("mes_celebracao")))

In [11]:
# contrats_analise = contrats_analise.with_columns(
#     pl.when(pl.col('indicator_1016')==0).then(pl.lit(1)).otherwise(pl.lit(0)).alias("indicator_1016_contrario"),
# )

Variáveis de controlo:

- Preço contratual
- Ano da Celebração do contrato
- Tipo de contrato
- Mês ???



Variável de interesse:
- Número de palavras no objeto contratual

In [147]:
contrats_analise_pd = contrats_analise.to_pandas()

In [148]:
# tentar perceber como ultrapassar esta questão do ano de celebração ser tão diferente
contrats_analise_pd["ano_celebracao"].value_counts()

ano_celebracao
2023    42714
2021    38907
2022    38516
2020    30083
2019    28713
2018    22552
Name: count, dtype: int64

In [149]:
contrats_analise_pd = contrats_analise_pd[(contrats_analise_pd["ano_celebracao"]==2022) | (contrats_analise_pd["ano_celebracao"]==2023)]

##### algumas questões para analisar a espec do modelo

In [98]:
contrats_analise_pd[contrats_analise_pd["Tipo(s) de contrato_LIMPO"]=="concessão de obras públicas"]["indicator_1016_contrario"].value_counts()

indicator_1016_contrario
0    74
1     3
Name: count, dtype: int64

In [99]:
contrats_analise_pd[contrats_analise_pd["Tipo(s) de contrato_LIMPO"]=="aquisição de bens móveis"]["indicator_1016_contrario"].value_counts()

indicator_1016_contrario
0    101566
1      9255
Name: count, dtype: int64

In [100]:
contrats_analise_pd[contrats_analise_pd["Tipo(s) de contrato_LIMPO"]=="aquisição de serviços"]["indicator_1016_contrario"].value_counts()

indicator_1016_contrario
0    73057
1     1394
Name: count, dtype: int64

In [101]:
contrats_analise_pd[contrats_analise_pd["Tipo(s) de contrato_LIMPO"]=="empreitadas de obras públicas"]["indicator_1016_contrario"].value_counts()

indicator_1016_contrario
0    13583
1      239
Name: count, dtype: int64

In [102]:
contrats_analise_pd[contrats_analise_pd["Tipo(s) de contrato_LIMPO"]=="locação de bens móveis"]["indicator_1016_contrario"].value_counts()

indicator_1016_contrario
0    2072
1     116
Name: count, dtype: int64

In [103]:
contrats_analise_pd[contrats_analise_pd["Tipo(s) de contrato_LIMPO"]=="concessão de serviços públicos"]["indicator_1016_contrario"].value_counts()

indicator_1016_contrario
0    111
1     15
Name: count, dtype: int64

In [109]:
contrats_analise_pd["ano_celebracao"].value_counts()

ano_celebracao
2023    42714
2021    38907
2022    38516
2020    30083
2019    28713
2018    22552
Name: count, dtype: int64

In [112]:
contrats_analise_pd[contrats_analise_pd["ano_celebracao"]==2023]["indicator_1016_contrario"].value_counts()

indicator_1016_contrario
0    38600
1     4114
Name: count, dtype: int64

In [113]:
contrats_analise_pd[contrats_analise_pd["ano_celebracao"]==2022]["indicator_1016_contrario"].value_counts()

indicator_1016_contrario
0    35249
1     3267
Name: count, dtype: int64

In [114]:
contrats_analise_pd.columns

Index(['N.º Procedimento (ID BASE)', 'N.º Contrato',
       'model_confidence_logistic', 'model_prediction_logistic',
       'flag_1011_model_logistic', 'model_confidence_xgboost',
       'model_prediction_xgboost', 'flag_1011_model_xgboost', 'flag_1011_anom',
       'flag_1011_dist', 'objeto_len', 'Tipo(s) de contrato_LIMPO',
       'Objeto_LIMPO', 'Objeto_LIMPO_2', 'Objeto', 'Tipo de procedimento',
       'Tipo de procedimento_right', 'Contratação Excluída', 'Objeto_right',
       'Data Decisão Contratar', 'Data da decisão adjudicação',
       'Data Celebração', 'Tipo(s) de contrato',
       'Local de execução das principais prestações objeto do contrato',
       'Preço BASE (€)', 'Preço Contratual (€)',
       'Entidade(s) Adjudicante(s) - NIF', 'Número de Ordem do Lote',
       'indicator_1016', 'ano_celebracao', 'mes_celebracao',
       'indicator_1016_contrario'],
      dtype='object')

In [123]:
contrats_analise_pd[contrats_analise_pd['flag_1011_anom']==0]["indicator_1016_contrario"].value_counts()

indicator_1016_contrario
0    190441
1     11022
Name: count, dtype: int64

##### Continuação

In [150]:
contrats_analise_pd["log_preco"] = np.log1p(contrats_analise_pd["Preço Contratual (€)"])
contrats_analise_pd = pd.get_dummies(contrats_analise_pd, columns=["ano_celebracao", "Tipo(s) de contrato", "Tipo de procedimento"], drop_first=True)

In [151]:
# só para verificar que não há duplicados indesejados
contrats_analise_pd[contrats_analise_pd.duplicated(subset=["N.º Procedimento (ID BASE)", "N.º Contrato", "Número de Ordem do Lote"])]

,N.º Procedimento (ID BASE),N.º Contrato,model_confidence_logistic,model_prediction_logistic,flag_1011_model_logistic,model_confidence_xgboost,model_prediction_xgboost,flag_1011_model_xgboost,flag_1011_anom,flag_1011_dist,...,Tipo de procedimento_Concurso limitado por prévia qualificação,Tipo de procedimento_Concurso público,Tipo de procedimento_Concurso público simplificado,Tipo de procedimento_Consulta Prévia,Tipo de procedimento_Consulta Prévia Simplificada,"Tipo de procedimento_Consulta prévia ao abrigo do artigo 7º da Lei n.º 30/2021, de 21.05",Tipo de procedimento_Contratação excluída II,Tipo de procedimento_Procedimento de negociação,Tipo de procedimento_Serviços sociais e outros serviços específicos,Tipo de procedimento_Setores especiais ? isenção parte II


In [152]:
# dropar missing values no preço
contrats_analise_pd.dropna(subset="log_preco", inplace=True)

In [153]:
contrats_analise_pd["cri"] = (contrats_analise_pd["flag_1011_dist"] + contrats_analise_pd["flag_1011_model_logistic"] + contrats_analise_pd["flag_1011_model_xgboost"])/3

In [160]:
import statsmodels.api as sm

# Target
y = contrats_analise_pd["indicator_1016_contrario"]   # 0/1

# Features (exemplo)
X = contrats_analise_pd[
    ["log_preco"] +
    [c for c in contrats_analise_pd.columns if c.startswith("ano_celebracao_")] +
    [c for c in contrats_analise_pd.columns if c.startswith("Tipo(s) de contrato_")] + 
    # [c for c in contrats_analise_pd.columns if c.startswith("Tipo de procedimento_")] + 
    #["has_caderno"] +
    # ["caderno_pages"] +
    ["flag_1011_model_logistic"] +
    ["flag_1011_dist"] +
    # [c for c in contrats_analise_pd.columns if c.startswith("flag_1011")] +
    ["objeto_len"]
    # ["cri"]
    # ["flag_1011"]
]

In [161]:
X.drop(columns=["Tipo(s) de contrato_LIMPO"], inplace=True)

/tmp/ipykernel_3460856/1980204724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=["Tipo(s) de contrato_LIMPO"], inplace=True)


In [156]:
## o objetivo é ver os duplicados do número do procedimento e do nº do contrato
aux = contrats_analise.group_by(["N.º Procedimento (ID BASE)", "N.º Contrato"]).len()
contrats_analise \
    .group_by(["N.º Procedimento (ID BASE)", "N.º Contrato"]) \
    .agg([
        pl.col("indicator_1016_contrario").n_unique().alias("indicator_1016_contrario_unique"),
        pl.col("flag_1011_dist").n_unique().alias("flag_dist_unique"),
        pl.col("flag_1011_model_logistic").n_unique().alias("flag_model_logistic")
    ]).filter(pl.col("indicator_1016_contrario_unique")>1)

N.º Procedimento (ID BASE),N.º Contrato,indicator_1016_contrario_unique,flag_dist_unique,flag_model_logistic
str,str,u32,u32,u32
"""6375152""","""10349418""",2,1,1
"""4607349""","""9113690""",2,1,1
"""6130203""","""9689875""",2,1,1
"""5348350""","""7787374""",2,1,1
"""5967667""","""9462529""",2,1,1
…,…,…,…,…
"""6706238""","""10401996""",2,1,1
"""5760212""","""9311326""",2,1,1
"""6749147""","""10582469""",2,1,1


In [162]:
X_sm = sm.add_constant(X.to_numpy(dtype=float), has_constant="add")
y_sm = y.to_numpy(dtype=float)

res = sm.Logit(y_sm, X_sm).fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.280997
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                81230
Model:                          Logit   Df Residuals:                    81219
Method:                           MLE   Df Model:                           10
Date:                Tue, 27 Jan 2026   Pseudo R-squ.:                 0.07729
Time:                        23:38:54   Log-Likelihood:                -22825.
converged:                       True   LL-Null:                       -24737.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8561      0.066    -58.669      0.000      -3.985      -3.727
x1             0.2219      0.

In [163]:
X_clean = X.copy()
X_clean = X_clean.astype(float)
y_clean = y.astype(float)

res = sm.Logit(
    y_clean,
    sm.add_constant(X_clean)
).fit(cov_type="HC3")

print(res.summary())

Optimization terminated successfully.
         Current function value: 0.280997
         Iterations 7
                              Logit Regression Results                              
Dep. Variable:     indicator_1016_contrario   No. Observations:                81230
Model:                                Logit   Df Residuals:                    81219
Method:                                 MLE   Df Model:                           10
Date:                      Tue, 27 Jan 2026   Pseudo R-squ.:                 0.07729
Time:                              23:38:55   Log-Likelihood:                -22825.
converged:                             True   LL-Null:                       -24737.
Covariance Type:                        HC3   LLR p-value:                     0.000
                                                         coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

In [126]:
contrats_analise.group_by("ano_celebracao").agg(
    pl.len().alias("n"),
    pl.col("indicator_1016_contrario").mean()
)

ano_celebracao,n,indicator_1016_contrario
i32,u32,f64
2020,30083,0.019978
2022,38516,0.084822
2018,22552,0.000044
2023,42714,0.096315
2019,28713,0.000592
2021,38907,0.077672


In [127]:
contrats_analise.group_by("Tipo(s) de contrato").agg(
    pl.len().alias("n"),
    pl.col("indicator_1016_contrario").mean()
)


Tipo(s) de contrato,n,indicator_1016_contrario
str,u32,f64
"""Concessão de obras públicas""",77,0.038961
"""Locação de bens móveis""",2188,0.053016
"""Empreitadas de obras públicas""",13822,0.017291
"""Concessão de serviços públicos""",126,0.119048
"""Aquisição de serviços""",74451,0.018724
"""Aquisição de bens móveis""",110821,0.083513


In [130]:
contrats_analise.group_by(["ano_celebracao", "Tipo(s) de contrato"]).agg(
    pl.len().alias("n"),
    pl.col("indicator_1016_contrario").n_unique()
).filter(pl.col("n")==1)

ano_celebracao,Tipo(s) de contrato,n,indicator_1016_contrario
i32,str,u32,u32


In [125]:
# X = DataFrame with predictors only
corr = X.corr(method="pearson")

# view

# optional: nicer formatting
corr.round(3)

,log_preco,ano_celebracao_2019,ano_celebracao_2020,ano_celebracao_2021,ano_celebracao_2022,ano_celebracao_2023,Tipo(s) de contrato_Aquisição de serviços,Tipo(s) de contrato_Concessão de obras públicas,Tipo(s) de contrato_Concessão de serviços públicos,Tipo(s) de contrato_Empreitadas de obras públicas,Tipo(s) de contrato_Locação de bens móveis,flag_1011_model_logistic,flag_1011_dist,objeto_len
log_preco,1.000,-0.020,0.001,-0.044,0.015,0.057,0.109,0.009,0.004,0.242,0.036,0.056,0.010,0.170
ano_celebracao_2019,-0.020,1.000,-0.171,-0.199,-0.198,-0.211,0.044,0.000,0.001,0.019,0.005,0.007,-0.000,-0.000
ano_celebracao_2020,0.001,-0.171,1.000,-0.205,-0.204,-0.217,-0.008,0.001,-0.003,0.022,-0.004,0.004,0.001,-0.011
ano_celebracao_2021,-0.044,-0.199,-0.205,1.000,-0.238,-0.254,-0.041,0.000,0.000,0.006,-0.003,-0.004,-0.005,-0.022
ano_celebracao_2022,0.015,-0.198,-0.204,-0.238,1.000,-0.252,-0.014,0.001,0.002,-0.035,0.000,-0.008,-0.003,0.014
ano_celebracao_2023,0.057,-0.211,-0.217,-0.254,-0.252,1.000,-0.011,-0.002,0.001,-0.021,0.002,-0.005,0.004,0.020
Tipo(s) de contrato_Aquisição de serviços,0.109,0.044,-0.008,-0.041,-0.014,-0.011,1.000,-0.015,-0.019,-0.208,-0.080,0.013,0.037,0.194
Tipo(s) de contrato_Concessão de obras públicas,0.009,0.000,0.001,0.000,0.001,-0.002,-0.015,1.000,-0.000,-0.005,-0.002,0.033,-0.001,0.002
Tipo(s) de contrato_Concessão de serviços públicos,0.004,0.001,-0.003,0.000,0.002,0.001,-0.019,-0.000,1.000,-0.007,-0.003,-0.001,-0.001,0.006
Tipo(s) de contrato_Empreitadas de obras públicas,0.242,0.019,0.022,0.006,-0.035,-0.021,-0.208,-0.005,-0.007,1.000,-0.028,0.125,-0.010,0.066
